In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import matplotlib.pyplot as plt


In [2]:
class DataSet(Dataset):
    def __init__(self, img_path, sigma, transforms=None):
        self.img_path = img_path
        self.transforms = transforms
        self.imgs = os.listdir(img_path)
        self.sigma = sigma

    def __len__(self):
        return len(self.imgs)
    
    def __getitem__(self, idx):
        path_file = os.path.join(self.img_path, self.imgs[idx])
        clean = Image.open(path_file).convert("RGB")
        
        transform = transforms.Compose([
            transforms.Resize((256, 256)),
            transforms.ToTensor()  
        ])
        
        clean = transform(clean)  
        noise = torch.randn(clean.shape) * self.sigma  
        noisy = torch.clamp(clean + noise, 0., 1.) 

        
        return noisy, clean


In [3]:
class Autoencoder(nn.Module):
        def __init__(self):
            super(Autoencoder, self).__init__()
            
            # Encoder
            self.encoder = nn.Sequential(
                nn.Conv2d(3, 64, kernel_size=3, stride=2, padding=1),  
                nn.ReLU(True),
                nn.Conv2d(64, 128, kernel_size=3, stride=2, padding=1),  
                nn.ReLU(True),
                nn.Conv2d(128, 256, kernel_size=3, stride=2, padding=1),  
                nn.ReLU(True),
                nn.Conv2d(256, 512, kernel_size=3, stride=2, padding=1), 
                nn.ReLU(True),

                
            )
            
            # Decoder
            self.decoder = nn.Sequential(

                nn.ConvTranspose2d(512, 256, kernel_size=3, stride=2, padding=1, output_padding=1),  
                nn.ReLU(True),
                nn.ConvTranspose2d(256, 128, kernel_size=3, stride=2, padding=1, output_padding=1),  
                nn.ReLU(True),
                nn.ConvTranspose2d(128, 64, kernel_size=3, stride=2, padding=1, output_padding=1),  
                nn.ReLU(True),
                nn.ConvTranspose2d(64, 3, kernel_size=3, stride=2, padding=1, output_padding=1), 
                nn.Sigmoid()
            )

        def forward(self, x):
            x = self.encoder(x)
            x = self.decoder(x)
            return x

In [ ]:
# Hyperparameters
num_epochs = 15
learning_rate = 0.001
batch_size = 32
sigma = 0.01
img_path = '/home/mint/Downloads/img/color' 



dataset = DataSet(img_path=img_path, sigma=sigma)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)


model = Autoencoder().cuda()
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training Loop
for epoch in range(num_epochs):
    for data in dataloader:
        noisy_imgs, clean_imgs = data
        noisy_imgs = noisy_imgs.cuda()
        clean_imgs = clean_imgs.cuda()

        outputs = model(noisy_imgs)
        loss = criterion(outputs, clean_imgs)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

print('Training complete.')


In [ ]:
noisy_images, clean_images = next(iter(dataloader))
noisy_image = noisy_images[0].unsqueeze(0).cuda()


with torch.no_grad():
    denoised_image = model(noisy_image)


noisy_image = noisy_image.squeeze().permute(1, 2, 0).cpu().numpy()
denoised_image = denoised_image.squeeze().permute(1, 2, 0).cpu().numpy()


plt.figure(figsize=(10, 5))

plt.subplot(1, 2, 1)
plt.imshow(noisy_image)
plt.title('Noisy Image')
plt.axis('off')

plt.subplot(1, 2, 2)
plt.imshow(denoised_image)
plt.title('Denoised Image')
plt.axis('off')

plt.show()